GridSearch (choose parameter to try, lr, dropout)
early stopping

Deactivating callbacks can be especially useful when you do a parameter search (say with sklearn GridSearchCV). If, for instance, you use a callback for learning rate scheduling (e.g. via LRScheduler) and want to test its usefulness, you can compare the performance once with and once without the callback.

Implement confusion matrix and progress bar

key_ignore=confusion_matrix put in realization

implement early stopping

implement model save

implement gridSearch

implement space_cnn

implement long core for cnn

output labels


In [1]:
import os
import numpy as np
import scipy.io

In [2]:
mods = ['BPSK', 'DQPSK', 'GFSK', 'GMSK', 'OQPSK',
        'PAM4', 'PAM8', 'PSK8', 'QAM16', 'QAM64', 'QPSK']
class_num = len(mods)

In [3]:
def import_from_mat(mods, fpath, size):

    # import data of mat format
    data = scipy.io.loadmat(fpath)
    features = []
    labels = []
    for mod in mods:
        real = np.array(data[mod].real[:size])
        imag = np.array(data[mod].imag[:size])
        signal = np.concatenate([real, imag], axis=1)
        features.append(signal)
        labels.append(mods.index(mod) * np.ones([size, 1]))

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

In [4]:
features, labels = import_from_mat(mods,
                                   "D:/Archive/0006/"
                                   "batch100000_symbols128_sps8_baud1_snr5.dat",
                                   100000
                                  )

In [15]:
features = features.astype(np.float32)
labels = labels.astype(np.int64)

In [16]:
X = features
y = labels.reshape(-1)

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class Discriminator(nn.Module):
    """Define the model"""

    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(2, 256, 3, padding=1),  # batch, 256, 1024
            nn.BatchNorm1d(256),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 80, 3, padding=1),  # batch, 80, 1024
            nn.BatchNorm1d(80),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(80 * 1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.6)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, class_num),
            nn.ReLU()
        )
        
    def forward(self, x, **kwargs):
        x = x.reshape((x.size(0), 2, -1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [19]:
from skorch import NeuralNetClassifier
from skorch.callbacks import EpochScoring, PrintLog, ProgressBar
from sklearn.metrics import confusion_matrix
from skorch.utils import data_from_dataset
import sys

In [20]:
class Score_ConfusionMatrix(EpochScoring):
    def on_epoch_end(self, net, dataset_train, dataset_valid, **kwargs):
        
        EpochScoring.on_epoch_end(self, net, dataset_train, dataset_valid)
        
        X_test, y_test = data_from_dataset(dataset_valid)
        y_pred = net.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        history = net.history
        history.record("confusion_matrix", cm)

In [21]:
class Print_Score_CM(PrintLog):
    
    def on_epoch_end(self, net, **kwargs):
        PrintLog.on_epoch_end(self, net)
        history = net.history
        print(history[-1, "confusion_matrix"])
        sys.stdout.flush()

In [22]:
net = NeuralNetClassifier(
    Discriminator,
    max_epochs=20,
    lr=0.01,
    device='cuda',
    iterator_train__shuffle=True,
    iterator_valid__shuffle=False
)

score = Score_ConfusionMatrix(scoring="accuracy", lower_is_better=False)
pt = Print_Score_CM(keys_ignored="confusion_matrix")
net.set_params(callbacks__valid_acc=score)
net.set_params(callbacks__print_log=pt)

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.Discriminator'>,
)

In [23]:
net.fit(X,y)

  epoch    accuracy    train_loss    valid_loss       dur
-------  ----------  ------------  ------------  --------
      1      0.8927        0.3259        0.2037  336.4606
[[20000     0     0     0     0     0     0     0     0     0     0]
 [    0 15545     0     0    12     0     0   113     0     0  4330]
 [    0     0 20000     0     0     0     0     0     0     0     0]
 [    0     0     0 20000     0     0     0     0     0     0     0]
 [    0     1     0     0 19838     0     0     0     0     0   161]
 [    0     0     0     0     0 20000     0     0     0     0     0]
 [    0     0     0     0     0     0 20000     0     0     0     0]
 [    0  1095     0     0    22     0     0  1110     0     0 17773]
 [    0     0     0     0     0     0     0     0 20000     0     0]
 [    0     0     0     0     0     0     0     0     0 20000     0]
 [    0    71     0     0    33     0     0     0     0     0 19896]]


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Discriminator(
    (conv1): Sequential(
      (0): Conv1d(2, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv1d(256, 80, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (fc1): Sequential(
      (0): Linear(in_features=81920, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.6)
    )
    (fc2): Sequential(
      (0): Linear(in_features=256, out_features=11, bias=True)
      (1): ReLU()
    )
  ),
)